In [1]:
import tensorflow as tf
import numpy as np
from PIL import Image,ImageOps 
import os
from tqdm import tqdm
import pandas as pd
from keras.utils import to_categorical
from tensorflow.keras.utils import Sequence
from matplotlib import pyplot as plt
from sklearn import model_selection
import seaborn as sns
from keras.models import Sequential
from sklearn.metrics import confusion_matrix
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import gc
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model

# Check if GPU is available
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

# List available devices
print("Available devices:", tf.config.list_physical_devices())

/Users/sufi/Desktop/Term 7/CV/drowsiness_detection/Driver-Fatigue-Project/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


Num GPUs Available:  1
Available devices: [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [2]:
# Check the mode of image
sample_image_directory = '../Datasets/mrlEyes/s0001/s0001_00001_0_0_0_0_0_01.png'
image = image = Image.open(sample_image_directory)
print("Image mode: ",image.mode)

# All the images are in grayscale mode (i.e L 8-bit grayscale)

Image mode:  L


In [3]:
# Initialize lists to store image data and labels
img_path_list = []
label = []

# Define the path to the directory containing the images
directory = '../Datasets/mrlEyes'

# Loop through each subdirectory and process images
for i in tqdm(os.listdir(directory), desc="Processing directories"):
    # Construct the full path to each subdirectory in 'directory'
    sub_path = os.path.join(directory, i)
    if os.path.isdir(sub_path):
        # Use tqdm for tracking progress within each subdirectory
        for j in tqdm(os.listdir(sub_path), desc=f"Processing images in {i}", leave=False):
            img_path = os.path.join(sub_path, j)
            img_path_list.append(img_path)

            # Assign labels based on filename convention
            if len(j) > 16 and j[16] == '1':  # Ensure index 16 exists in filename
                label.append(1)  # Drowsy
            else:
                label.append(0)  # Non-Drowsy

label = np.array(label)

print("Label shape:", label.shape)


Processing directories: 100%|██████████| 40/40 [00:00<00:00, 147.12it/s]

Label shape: (84898,)


In [4]:
# Split into train, validation, and test sets
train_paths, temp_paths, train_labels, temp_labels = train_test_split(img_path_list, label, test_size=0.3, random_state=42)
val_paths, test_paths, val_labels, test_labels = train_test_split(temp_paths, temp_labels, test_size=0.5, random_state=42)

In [5]:
# Define an augmentation generator
augmentation_generator = ImageDataGenerator(
        rotation_range=15,          # Slight rotations
        brightness_range=[0.8, 1.2], # Adjust brightness
        horizontal_flip=True,       # Flip horizontally
        zoom_range=0.1,             # Slight zoom in/out
        width_shift_range=0.1,      # Horizontal shifts
        height_shift_range=0.1,     # Vertical shifts
        shear_range=10,             # Shearing distortions
        fill_mode='nearest'         # Fill missing pixels
)

In [6]:
# Define a custom DataGenerator class
class MRLDataGenerator(Sequence):
    def __init__(self, file_paths, labels, batch_size=32, target_size=(224, 224),shuffle = True, augment = False):
        self.file_paths = file_paths
        self.labels = labels
        self.batch_size = batch_size
        self.target_size = target_size
        self.shuffle = shuffle
        self.augment = augment
        self.on_epoch_end()

    def __len__(self):
        # Calculate the number of batches per epoch
        return int(np.ceil(len(self.file_paths) / self.batch_size))

    def __getitem__(self, index):
        # Fetch batch data
        batch_paths = self.file_paths[index * self.batch_size:(index + 1) * self.batch_size]
        batch_labels = self.labels[index * self.batch_size:(index + 1) * self.batch_size]
        
        images = []
        for file_path in batch_paths:
            # Load and preprocess the image
            image = load_img(file_path, target_size=self.target_size, color_mode='rgb')
            image = img_to_array(image) / 255.0  # Normalize to [0, 1]
            if self.augment:
                image = augmentation_generator.random_transform(image)
            images.append(image)
        
        return np.array(images), np.array(batch_labels)
    
    def on_epoch_end(self):
        self.indexes = np.arange(len(self.file_paths))
        if self.shuffle:
            np.random.shuffle(self.indexes)

# Batch size
batch_size = 32

# Create generators
train_generator = MRLDataGenerator(train_paths, train_labels, batch_size=batch_size)
val_generator = MRLDataGenerator(val_paths, val_labels, batch_size=batch_size)
test_generator = MRLDataGenerator(test_paths, test_labels, batch_size=batch_size)


In [7]:
# Load MobileNetV2 for transfer learning
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False  # Freeze base model
# base_model.summary()




2024-11-18 15:11:41.655077: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1
2024-11-18 15:11:41.655160: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2024-11-18 15:11:41.655180: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2024-11-18 15:11:41.655236: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-11-18 15:11:41.655256: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [8]:
# Add custom classification layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(1, activation='sigmoid')(x)

# Define the model
model = Model(inputs=base_model.input, outputs=predictions)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=10,
    steps_per_epoch=len(train_generator),
    validation_steps=len(val_generator)
)

# Evaluate the model
test_loss, test_accuracy = model.evaluate(test_generator, steps=len(test_generator))
print(f"Test Accuracy: {test_accuracy:.2f}")

Epoch 1/10


/Users/sufi/Desktop/Term 7/CV/drowsiness_detection/Driver-Fatigue-Project/.venv/lib/python3.9/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
2024-11-18 15:11:52.545705: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


1858/1858 ━━━━━━━━━━━━━━━━━━━━ 371s 198ms/step - accuracy: 0.9154 - loss: 0.2134 - val_accuracy: 0.9606 - val_loss: 0.1120
Epoch 2/10
1858/1858 ━━━━━━━━━━━━━━━━━━━━ 0s 56us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 3/10


2024-11-18 15:18:02.645572: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2024-11-18 15:18:02.645590: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[StatefulPartitionedCall/adam/Add_8/_20]]
2024-11-18 15:18:02.645609: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 458268986106212237
2024-11-18 15:18:02.645613: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 16582438471703293250
2024-11-18 15:18:02.645622: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 11825175914835176135
2024-11-18 15:18:02.645629: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 9742942922

1858/1858 ━━━━━━━━━━━━━━━━━━━━ 372s 200ms/step - accuracy: 0.9544 - loss: 0.1233 - val_accuracy: 0.9597 - val_loss: 0.1004
Epoch 4/10
1858/1858 ━━━━━━━━━━━━━━━━━━━━ 0s 45us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 5/10


...

1858/1858 ━━━━━━━━━━━━━━━━━━━━ 373s 201ms/step - accuracy: 0.9641 - loss: 0.0988 - val_accuracy: 0.9662 - val_loss: 0.0861
Epoch 8/10
1858/1858 ━━━━━━━━━━━━━━━━━━━━ 0s 59us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 9/10


2024-11-18 15:36:49.097953: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2024-11-18 15:36:49.097972: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 3250343175779298604
2024-11-18 15:36:49.097980: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[StatefulPartitionedCall/adam/Add_4/_28]]
2024-11-18 15:36:49.097984: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 458268986106212237
2024-11-18 15:36:49.097987: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 11825175914835176135
2024-11-18 15:36:49.097991: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 16053860078

1858/1858 ━━━━━━━━━━━━━━━━━━━━ 374s 201ms/step - accuracy: 0.9636 - loss: 0.0953 - val_accuracy: 0.9724 - val_loss: 0.0714
Epoch 10/10
1858/1858 ━━━━━━━━━━━━━━━━━━━━ 0s 53us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00


2024-11-18 15:43:03.568250: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2024-11-18 15:43:03.568269: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 16582438471703293250
2024-11-18 15:43:03.568286: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 9742942922504950146
2024-11-18 15:43:03.568291: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 8246542230632731218
2024-11-18 15:43:03.568297: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 12354904682724877370
2024-11-18 15:43:03.568301: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 5274594298325706396
2024-11-18 15:43:03.568304: I tensorflow/core/framework/local_rendezv

398/398 ━━━━━━━━━━━━━━━━━━━━ 61s 154ms/step - accuracy: 0.9721 - loss: 0.0762
Test Accuracy: 0.97


In [9]:
model.save('../content/model2.h5', overwrite=True)